In [ ]:
%matplotlib inline

###Two versions of dust file

1. oh5_kappa includes opacity per gram of gas and wavelength in micron
2. sigma_oh5.ref includes absorption and scattering cross section per dust grain, anisotropy factor, and wavelength in micron

## Hyperion dust input:
  [nu, albedo, chi, g, p_lin_max]
  * nu ($\nu$): Frequency
  * albedo: $\sigma_{\rm scat}/(\sigma_{\rm scat}+\sigma_{\rm abs})$.  This quantity is dimensionless.
  * chi ($\chi_{\nu}$): The mass extinction coefficient, opacity per gram of dust.
  * g: The anisotropy scattering factor.
  * p_lin_max: something to do with polarization.  No polarization is assumed, therefore set to zero.

$$\rho_{\rm d} \kappa_{\rm d} = n_{\rm d}(\sigma_{\rm abs}+\sigma_{\rm scat})$$ 

$$\kappa_{\rm d} = (\sigma_{\rm abs}+\sigma_{\rm scat})\frac{n_{\rm d}}{\rho_{\rm d}} = 100 \kappa_{\rm g}$$ 

$$\kappa_{\rm g} = (\sigma_{\rm abs}+\sigma_{\rm scat})\frac{n_{\rm d}}{100\rho_{\rm d}} = (\sigma_{\rm abs}+\sigma_{\rm scat})\frac{n_{\rm d}}{\rho_{\rm g}}$$ 

and by definition
$$\rho_{\rm g} = \mu m_{\rm H}n_{\rm g}, \quad \mu = 2.29$$
so that
$$\kappa_{\rm g} = (\sigma_{\rm abs}+\sigma_{\rm scat})\frac{n_{\rm d}}{\mu m_{\rm H}n_{\rm g}}$$
$$\kappa_{\rm d} = (\sigma_{\rm abs}+\sigma_{\rm scat})\frac{100n_{\rm d}}{\rho_{\rm g}} = (\sigma_{\rm abs}+\sigma_{\rm scat})\frac{100n_{\rm d}}{\mu m_{\rm H}n_{\rm g}}$$
with the communication with Neal
$$\frac{n_{\rm d}}{\mu n_{\rm g}} =  1.1015 \times 10^{-14}$$

Appaerently more complicate than that.  Adopting the ratio between two files from now on.
$$\frac{\kappa_{\rm g}}{\sigma_{\rm d}} = 1.039 \times 10^{12}$$

$$\frac{\kappa_{\rm d}}{\sigma_{\rm d}} = 1.039 \times 10^{14}$$

In [18]:
import astropy.constants as const
c = const.c.cgs.value
mh = const.m_p.cgs.value + const.m_e.cgs.value

import numpy as np
oh5_kappa = np.genfromtxt('/Users/yaolun/Copy/Dustfiles/OH/oh5.kappa', skip_header=1).T  # This is opacity per gram of gas
oh5_sigma = np.genfromtxt('/Users/yaolun/Copy/Dustfiles/sigma.oh5.ref', skip_header=2, skip_footer=1).T
# This one has absorption and scattering cross section and g value with wavelength 

# oh5_sigma has more wavelength channels

nu = c/oh5_sigma[3,:]*1e4
# nu = oh5_sigma[3,:]
albedo = oh5_sigma[1,:]/(oh5_sigma[0,:]+oh5_sigma[1,:])
chi = (oh5_sigma[0,:]+oh5_sigma[1,:]) * 1.0395087779657002e14
g = oh5_sigma[2,:]

# sort by frequency
ind = np.argsort(nu)
nu = nu[ind]
albedo = albedo[ind]
chi = chi[ind]
g = g[ind]

foo = open('/Users/yaolun/Copy/Dustfiles/oh5_hyperion.txt','w')
for i in range(0, len(nu)):
    foo.write('%e \t %e \t %e \t %e \n' % (nu[i],albedo[i],chi[i],g[i]))
foo.close()


#Test out Ormel et al. 2009 model

In [2]:
def opac_file(age_ind):
    from astropy.io import ascii
    # read in Ormel et al. 2011
    dust_ormel = ascii.read(opac_file(4), header_start=None, names=['lambda','kappa-ext','kappa-sca','kappa-back','gsca'])
    return '/Users/yaolun/Copy/dust_model/Ormel2011/(ic-sil,ic-gra)'+str(age_ind)+'opc.txt'

In [4]:
import numpy as np

import matplotlib.pyplot as plt

# read in Ormel et al. 2011
dust_ormel = ascii.read(opac_file(4), header_start=None, names=['lambda','kappa-ext','kappa-sca','kappa-back','gsca'])